# Google Agent Development Kit (ADK) - Fundamentals

## Introduction

The **Google Agent Development Kit (ADK)** is Google's framework for building AI agents that can:
- Execute multi-step workflows
- Use tools and APIs
- Communicate with other agents (Agent-to-Agent)
- Maintain state and memory
- Integrate with Google Cloud services

### What You'll Learn
- ADK architecture and core concepts
- Setting up ADK environment
- Building your first ADK agent
- ADK vs LangGraph comparison
- When to use ADK

## Part 1: What is Google ADK?

### Overview

Google ADK is designed for enterprise-grade agent development with:

- **Native Google Cloud Integration**: Seamless with Vertex AI, Cloud Functions, BigQuery
- **Agent-to-Agent Communication**: Built-in A2A protocol for multi-agent systems
- **Production-Ready**: Monitoring, logging, and deployment tools included
- **Scalability**: Designed for large-scale agent deployments

### Architecture

```
┌─────────────────────────────────────┐
│      Google ADK Architecture        │
├─────────────────────────────────────┤
│                                     │
│  ┌──────────────────────────────┐  │
│  │    Agent Runtime Engine      │  │
│  │  (Orchestration & Execution) │  │
│  └──────────┬───────────────────┘  │
│             │                       │
│   ┌─────────┼─────────┐            │
│   │         │         │            │
│   ▼         ▼         ▼            │
│ ┌────┐  ┌────┐  ┌──────┐          │
│ │LLM │  │Tools│  │Memory│          │
│ │API │  │     │  │Store │          │
│ └────┘  └────┘  └──────┘          │
│                                     │
│  ┌──────────────────────────────┐  │
│  │   A2A Communication Layer    │  │
│  │   (Agent-to-Agent Protocol)  │  │
│  └──────────────────────────────┘  │
│                                     │
│  ┌──────────────────────────────┐  │
│  │   Google Cloud Services      │  │
│  │  Vertex AI | Functions | etc │  │
│  └──────────────────────────────┘  │
└─────────────────────────────────────┘
```

## Part 2: Setup and Installation

### Prerequisites

```bash
# Install Google Cloud SDK
curl https://sdk.cloud.google.com | bash

# Install ADK
pip install google-cloud-adk

# Install dependencies
pip install google-cloud-aiplatform
pip install google-cloud-functions
```

### Authentication

```bash
# Set up application default credentials
gcloud auth application-default login

# Set project
gcloud config set project YOUR_PROJECT_ID
```

## Part 3: Core ADK Concepts

### 1. Agent Definition

In ADK, agents are defined with:
- **Skills**: What the agent can do (tools/functions)
- **Personality**: How the agent behaves
- **Memory**: What the agent remembers
- **Goals**: What the agent tries to achieve

In [ ]:
from google.cloud import adk
from google.cloud.adk import Agent, Skill, Memory

# Define a skill (tool)
class WeatherSkill(Skill):
    """Skill for getting weather information"""
    
    def __init__(self):
        super().__init__(
            name="get_weather",
            description="Get current weather for a location"
        )
    
    def execute(self, location: str) -> dict:
        """Execute the weather skill"""
        # In production, call actual weather API
        return {
            "location": location,
            "temperature": "72°F",
            "condition": "Sunny"
        }

# Define an agent
class WeatherAgent(Agent):
    """Agent that provides weather information"""
    
    def __init__(self):
        super().__init__(
            name="weather_assistant",
            description="I help users get weather information",
            skills=[WeatherSkill()],
            memory=Memory(type="conversation")
        )
    
    def process(self, user_input: str) -> str:
        """Process user request"""
        # Agent logic here
        return f"Processing: {user_input}"

# Create and use agent
agent = WeatherAgent()
print(f"Agent created: {agent.name}")
print(f"Skills: {[skill.name for skill in agent.skills]}")

### 2. Skills (Tools) in ADK

Skills are ADK's equivalent of tools in LangGraph.

In [ ]:
from google.cloud.adk import Skill, SkillParameter
from typing import List

class SearchSkill(Skill):
    """Skill for searching the web"""
    
    def __init__(self):
        super().__init__(
            name="search_web",
            description="Search the web for information",
            parameters=[
                SkillParameter(
                    name="query",
                    type="string",
                    description="Search query",
                    required=True
                ),
                SkillParameter(
                    name="max_results",
                    type="integer",
                    description="Maximum number of results",
                    required=False,
                    default=5
                )
            ]
        )
    
    def execute(self, query: str, max_results: int = 5) -> List[dict]:
        """Execute search"""
        # In production, call actual search API
        return [
            {"title": f"Result {i}", "url": f"https://example.com/{i}"}
            for i in range(max_results)
        ]
    
    def validate(self, **kwargs) -> bool:
        """Validate skill parameters"""
        query = kwargs.get("query")
        if not query or len(query) < 2:
            raise ValueError("Query must be at least 2 characters")
        return True

# Test skill
search_skill = SearchSkill()
results = search_skill.execute("AI agents", max_results=3)
print(f"Search results: {len(results)} items")

### 3. Memory Management

In [ ]:
from google.cloud.adk import Memory, MemoryType

# Conversation memory (short-term)
conversation_memory = Memory(
    type=MemoryType.CONVERSATION,
    max_tokens=2000,
    ttl=3600  # 1 hour
)

# Long-term memory (persisted)
longterm_memory = Memory(
    type=MemoryType.LONGTERM,
    backend="firestore",  # Google Firestore
    collection="agent_memories"
)

# Vector memory (semantic search)
vector_memory = Memory(
    type=MemoryType.VECTOR,
    backend="vertex_ai_matching_engine",
    dimension=768,
    index_endpoint="projects/PROJECT/locations/LOCATION/indexEndpoints/ENDPOINT"
)

# Use in agent
class SmartAgent(Agent):
    def __init__(self):
        super().__init__(
            name="smart_assistant",
            description="Agent with multiple memory types",
            memory=[
                conversation_memory,
                longterm_memory,
                vector_memory
            ]
        )

## Part 4: Building a Complete ADK Agent

In [ ]:
from google.cloud import adk
from google.cloud.adk import Agent, Skill, Memory, AgentConfig
from google.cloud.aiplatform import gapic as aiplatform_gapic

# 1. Define Skills
class CalculatorSkill(Skill):
    """Mathematical calculations"""
    
    def __init__(self):
        super().__init__(
            name="calculate",
            description="Perform mathematical calculations"
        )
    
    def execute(self, expression: str) -> float:
        try:
            # Safe evaluation
            result = eval(expression, {"__builtins__": {}}, {})
            return result
        except Exception as e:
            raise ValueError(f"Invalid expression: {e}")

# 2. Define Agent Configuration
config = AgentConfig(
    model="gemini-pro",
    temperature=0.7,
    max_tokens=1024,
    system_prompt="""You are a helpful assistant that can perform calculations.
    When asked to calculate, use the calculate skill.
    Always show your reasoning."""
)

# 3. Create Agent
class MathAgent(Agent):
    """Agent that helps with math"""
    
    def __init__(self):
        super().__init__(
            name="math_assistant",
            description="I help with mathematical calculations",
            skills=[CalculatorSkill()],
            config=config,
            memory=Memory(type="conversation")
        )
    
    async def run(self, user_input: str) -> str:
        """Process user request"""
        # Add to memory
        self.memory.add(role="user", content=user_input)
        
        # Get LLM response with skills
        response = await self.generate_response(
            messages=self.memory.get_messages(),
            skills=self.skills
        )
        
        # Execute any skill calls
        if response.skill_calls:
            for skill_call in response.skill_calls:
                result = self.execute_skill(skill_call)
                self.memory.add(role="skill", content=str(result))
            
            # Get final response
            response = await self.generate_response(
                messages=self.memory.get_messages()
            )
        
        # Add to memory and return
        self.memory.add(role="assistant", content=response.content)
        return response.content

# 4. Use Agent
async def main():
    agent = MathAgent()
    
    # Test
    response = await agent.run("What is 15 * 24?")
    print(f"Agent: {response}")
    
    # Follow-up (with memory)
    response = await agent.run("Now divide that by 3")
    print(f"Agent: {response}")

# Run
# await main()
print("Math agent example created")

## Part 5: ADK vs LangGraph

### Comparison Table

| Feature | Google ADK | LangGraph |
|---------|-----------|----------|
| **Provider** | Google Cloud | LangChain (open source) |
| **Integration** | Native GCP | Framework agnostic |
| **Agent-to-Agent** | Built-in A2A protocol | Custom implementation |
| **Deployment** | Cloud Functions, Cloud Run | Any platform |
| **Monitoring** | Cloud Logging, Monitoring | Custom |
| **Learning Curve** | Moderate | Low-Moderate |
| **Flexibility** | Opinionated | Very flexible |
| **Cost** | GCP pricing | Open source + LLM costs |
| **Best For** | Enterprise GCP users | General purpose |

### When to Use ADK

Choose Google ADK when:
- ✅ Already using Google Cloud Platform
- ✅ Need enterprise-grade monitoring/logging
- ✅ Building multi-agent systems (A2A)
- ✅ Want managed infrastructure
- ✅ Need tight Vertex AI integration

Choose LangGraph when:
- ✅ Need maximum flexibility
- ✅ Platform agnostic requirements
- ✅ Open source preference
- ✅ Custom control flow needed
- ✅ Smaller scale deployments

## Part 6: Deployment Patterns

### Deploy to Cloud Functions

In [ ]:
# main.py - Cloud Function entry point
"""
from google.cloud import adk
from flask import Request, jsonify

# Initialize agent
agent = MathAgent()

def handle_request(request: Request):
    \"\"\"Cloud Function entry point\"\"\"
    data = request.get_json()
    user_input = data.get('input')
    
    # Run agent
    response = await agent.run(user_input)
    
    return jsonify({
        'response': response,
        'agent': agent.name
    })
"""

# Deploy command:
# gcloud functions deploy math-agent \
#   --runtime python39 \
#   --trigger-http \
#   --entry-point handle_request \
#   --allow-unauthenticated

### Deploy to Cloud Run

In [ ]:
# app.py - Cloud Run service
"""
from google.cloud import adk
from flask import Flask, request, jsonify
import os

app = Flask(__name__)
agent = MathAgent()

@app.route('/chat', methods=['POST'])
async def chat():
    data = request.get_json()
    response = await agent.run(data['input'])
    return jsonify({'response': response})

if __name__ == '__main__':
    port = int(os.environ.get('PORT', 8080))
    app.run(host='0.0.0.0', port=port)
"""

# Dockerfile
"""
FROM python:3.9-slim
WORKDIR /app
COPY requirements.txt .
RUN pip install -r requirements.txt
COPY . .
CMD exec gunicorn --bind :$PORT --workers 1 --threads 8 app:app
"""

# Deploy:
# gcloud run deploy math-agent --source .

## Key Takeaways

- **Google ADK** is designed for enterprise-grade agent development
- **Native GCP integration** makes it ideal for Google Cloud users
- **Skills** are ADK's equivalent of tools/functions
- **Memory management** is built-in with multiple backend options
- **A2A protocol** enables sophisticated multi-agent systems
- **Deployment** is streamlined with Cloud Functions and Cloud Run

## Next Steps

In the next notebooks:
1. **Building ADK Agents** - Complete agent examples
2. **Agent-to-Agent (A2A)** - Multi-agent communication
3. **Production Patterns** - Monitoring, scaling, optimization

---

**Next**: [02_building_adk_agents.ipynb](./02_building_adk_agents.ipynb)